### Look A Like Modelling Dataset creation (dont need this in real world)

Hello all. Since this is beginning of the one of the people based modelling processes, LAL modelling, we will be looking into
creating a seed data, and a overall data. With Look a likes, we usually have big datasets consisting of unique users who may or may not have the same behavior which are defined over a particular list of variables (or features), which describe the user over some dimensions.

The goal of a look a like model is to find/ score a populus based on the maximum likelihood of behaviors they have over a seed data, something we want to find the users against.

This seed data are usually identifiers of a certian set of people, whose behaviors we are trying to map over certain behaviors or / dimensions and look for people with the most likelihood. The seed usually comes from a different analysis, or part of some sales / reach / leads / etc, independent to the behavior at first, we are tryinh to map it back to. This is usually part of a campaign data. Seed represents only a list of representatives / population, and some way to join it with our data. It is a small sample. 

However, the seed data is usualy restricted in terms of size and is ideally a subset of some other group of users. In a more realistic scenario, it could be for e.g., buyers for a car over a period which purchased over our marketing campaign, while the base could be of people who have showed intrest in buying our car but never bought the car. ( or could be an entire population of a locality)

We will then look into people who could have been the maximum likelihood of being potiential buyers of this car, what their most prominent features are, as well a decile (or any subset) or groups, which are the closest and will be targeted in the next ad-campaigns over any targeting platform. This helps in reducing costs for targeting (this is expensive you know, since it pays per user) as well as testing out any test control groups for any campaign analysis and with reporting. This could also be a prt of A/B testing

Any real world data could be purchased from Vendors, and usually go through a lot of preprocess to get us this info, e.g., data cleaning, removal of PII, mapping data to an hashed Identifier ID, hashing / creating dummy variables for x feilds, etc. This also includes getting it verified with a law unit in your org or through a data governacnce team.

##### Again, since the actual real world data cannot be obtained easily and could be very risky to deal with without governance ;-( , I will try to create an artificial dataset, something which closely resembles the real world collective data and seed so we can begin our analysis

Right now, we are doing the whole thing in python, but once i get to kno how to install and operate pyspark on local machine I might try that version too, as that is the ideally what any data science company likes to operate it on. Remember, python is non distributed single node working machine may not handle any big data of size of trillions(welcome to real world data!) efficiently.

However, lets get started !

We will be using the python's blob data function to create a dataset of size 30k, while keeping the seed to be 1.5k in size.
The number is features should be big tho(again following real world metadata you will usually deal with), like 150.

All the best! :-)

###### Note : This script is supposed to be run once, and it will save the data in your local. DO NOT RE-RUN!

In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_blobs

In [ ]:
data = make_blobs(n_samples=30000, n_features=80, cluster_std=20.2, random_state=123)

In [ ]:
data[0].shape # this is your data

In [ ]:
# Features for this dataset taken straight from chat GPT (I am out of ideas, sue me)
# On second thought -  please dont :'(

# Since features are important for our analysis, and in terms of handling model explainability, we will be keeping feature names as it is
# and not do any PCA's. (Important)

features=[('Feature_'+'0'*(3-len(str(i)))+str(i)) for i in range(1,81)]
print(len(features))

In [ ]:
# Identifiers for users, since they are lawfully restricted not to use / share / work with any PII information in raw form, or 
# as anything closely resembling it. (usually the case)
# Here I am generating a simpler artificial ids:
identifer_ids = [('INT'+'0'*(5-len(str(i)))+str(i)) for i in range(1,30001)]


# Create pandas data frame
df_univ=pd.DataFrame(data=data[0], columns=features, index=identifer_ids)
df_univ.insert(loc=0, column='User_ID' ,value=df_univ.index)
df_univ.reset_index(drop=True, inplace=True)

df_univ.head()

In [ ]:
## I am thinking of this data set as video watched hours for several shows / timings / recent watch behavior, etc. so keep it positive
df_univ[features]=df_univ[features].apply(lambda x:np.abs(x)*10)
df_univ.head()

In [ ]:
df_univ.shape

In [ ]:
df_univ.to_csv('movie_universe_features.csv', index=False)

In [ ]:
## Taking a stratified sample this time
seed_df = df_univ.apply(lambda r: sum([r[i] for i in features]) , axis=1)
seed_df.index=df_univ['User_ID']
seed_df = seed_df.sort_values(ascending=False)

## Take seed from 85 percentile
qntl = seed_df.quantile(.85)
seed_df = seed_df[seed_df>qntl]

if seed_df.shape[0]>1500:
    seed_df=seed_df.sample(n=1500, random_state=123,)
    
seed_df['User_ID'] = seed_df.index 
seed_df=pd.DataFrame(seed_df['User_ID']).reset_index(drop=True)
print(seed_df.shape)
print(df_univ.shape)

In [ ]:
# checking whether they can join or not
# .join works for joining by indexes, so leave that, and use merge. works the same
temp=pd.merge(seed_df, df_univ, how='left', on='User_ID')
print(temp.shape)

In [ ]:
# Save the seed IDs
#temp.head()
temp=None
seed_df.to_csv('seed.csv', index=False)

In [ ]:
#END

In [ ]:
#EDA
seed_df.head()

In [ ]:
# import seaborn as sns
#import matplotlib.pyplot as plt
#%matplotlib inline

In [ ]:
#from ydata_profiling import ProfileReport
import sweetviz as sv

#report = ProfileReport(df_univ, title='EDA of dataset')
report = sv.analyze(df_univ)

report.show_notebook()